In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from prophet import Prophet


In [3]:
df = pd.read_csv('C:/Users/Admin/Desktop/AQI_DATASET/datasets/city_hour.csv')
df.head()

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01 01:00:00,NaN,NaN,1.00,40.01,36.37,NaN,1.00,122.07,NaN,0.0,0.0,0.0,NaN,NaN
1,Ahmedabad,2015-01-01 02:00:00,NaN,NaN,0.02,27.75,19.73,NaN,0.02,85.90,NaN,0.0,0.0,0.0,NaN,NaN
2,Ahmedabad,2015-01-01 03:00:00,NaN,NaN,0.08,19.32,11.08,NaN,0.08,52.83,NaN,0.0,0.0,0.0,NaN,NaN
3,Ahmedabad,2015-01-01 04:00:00,NaN,NaN,0.30,16.45,9.20,NaN,0.30,39.53,153.58,0.0,0.0,0.0,NaN,NaN
4,Ahmedabad,2015-01-01 05:00:00,NaN,NaN,0.12,14.90,7.85,NaN,0.12,32.63,NaN,0.0,0.0,0.0,NaN,NaN


In [4]:
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)

df.fillna(method='ffill', inplace=True)

df_cleaned = df.drop(columns=['City', 'AQI_Bucket'])

df_cleaned.isnull().sum(), df_cleaned.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_2616\905221911.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


(PM2.5        665
 PM10       38274
 NO             0
 NO2            0
 NOx            0
 NH3        48192
 CO             0
 SO2            0
 O3             3
 Benzene        0
 Toluene        0
 Xylene         0
 AQI          680
 dtype: int64,
                      PM2.5  PM10    NO    NO2    NOx  NH3    CO     SO2  \
 Datetime                                                                  
 2015-01-01 01:00:00    NaN   NaN  1.00  40.01  36.37  NaN  1.00  122.07   
 2015-01-01 02:00:00    NaN   NaN  0.02  27.75  19.73  NaN  0.02   85.90   
 2015-01-01 03:00:00    NaN   NaN  0.08  19.32  11.08  NaN  0.08   52.83   
 2015-01-01 04:00:00    NaN   NaN  0.30  16.45   9.20  NaN  0.30   39.53   
 2015-01-01 05:00:00    NaN   NaN  0.12  14.90   7.85  NaN  0.12   32.63   
 
                          O3  Benzene  Toluene  Xylene  AQI  
 Datetime                                                    
 2015-01-01 01:00:00     NaN      0.0      0.0     0.0  NaN  
 2015-01-01 02:00:00     NaN   

In [5]:
df_cleaned.fillna(df_cleaned.median(), inplace=True)

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_cleaned)

df_scaled = pd.DataFrame(scaled_data, columns=df_cleaned.columns, index=df_cleaned.index)

df_scaled.head(100)


,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
Datetime,,,,,,,,,,,,,
2015-01-01 01:00:00,0.044491,0.072371,0.00198,0.080080,0.072943,0.025722,0.002006,0.610453,0.053677,0.000000,0.000000,0.00000,0.03296
2015-01-01 02:00:00,0.044491,0.072371,0.00002,0.055536,0.039570,0.025722,0.000040,0.429557,0.053677,0.000000,0.000000,0.00000,0.03296
2015-01-01 03:00:00,0.044491,0.072371,0.00014,0.038659,0.022222,0.025722,0.000160,0.264166,0.053677,0.000000,0.000000,0.00000,0.03296
2015-01-01 04:00:00,0.044491,0.072371,0.00058,0.032913,0.018451,0.025722,0.000602,0.197649,0.308615,0.000000,0.000000,0.00000,0.03296
2015-01-01 05:00:00,0.044491,0.072371,0.00022,0.029810,0.015744,0.025722,0.000241,0.163141,0.308615,0.000000,0.000000,0.00000,0.03296
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-05 00:00:00,0.044491,0.072371,0.00038,0.030911,0.027316,0.025722,0.000401,0.124131,0.085147,0.006385,0.014057,0.00000,0.03296
2015-01-05 01:00:00,0.044491,0.072371,0.00038,0.034394,0.025030,0.025722,0.000401,0.338185,0.074918,0.005481,0.014417,0.00166,0.03296
2015-01-05 02:00:00,0.044491,0.072371,0.00038,0.028729,0.017088,0.025722,0.000401,0.163341,0.076767,0.001245,0.006748,0.00000,0.03296


In [ ]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import matplotlib.pyplot as plt
import numpy as np
import logging
import os

# Reduce logging noise
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

def create_prophet_forecast_with_validation(df_scaled, target_column='AQI', periods=24*7):
    """
    Create and train a Prophet model with cross-validation and detailed diagnostics.
    Optimized for faster execution.
    """
    # Set Stan backend options for faster computation
    os.environ['STAN_SAMPLING_ARGS'] = 'method=optimize algorithm=lbfgs iter=1000'

    # Prepare data for Prophet - use a subset for faster computation
    prophet_df = df_scaled.reset_index()
    prophet_df = prophet_df.rename(columns={
        'Datetime': 'ds',
        target_column: 'y'
    })

    # Create and train Prophet model with minimal computation settings
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10,
        holidays_prior_scale=10,
        mcmc_samples=0,  # Disable MCMC
        interval_width=0.95,
        n_changepoints=20,  # Reduce changepoints further
        growth='linear'  # Simpler growth model
    )

    # Only add strongest correlating features
    correlation_threshold = 0.3  # Increased threshold for fewer regressors
    important_columns = [col for col in df_scaled.columns
                        if col != target_column
                        and df_scaled[col].corr(df_scaled[target_column]) > correlation_threshold]

    for column in important_columns[:3]:  # Limit to top 3 regressors
        model.add_regressor(column)

    # Fit the model
    model.fit(prophet_df)

    # Perform cross validation with fewer iterations
    cv_results = cross_validation(
        model,
        initial='365.25 days',
        period='90 days',     # Increased period for fewer iterations
        horizon='7 days',
        parallel="processes",
        disable_tqdm=True    # Disable progress bar
    )

    # Calculate performance metrics
    cv_metrics = performance_metrics(cv_results)

    # Create future dataframe
    future = model.make_future_dataframe(periods=periods, freq='H')

    # Add regressor values efficiently
    for column in important_columns[:3]:
        future[column] = prophet_df[column].iloc[-1]

    # Make forecast
    forecast = model.predict(future)

    return model, forecast, prophet_df, cv_metrics, cv_results

# Example usage
def run_forecast(df_scaled):
    # Optional: Use a subset of data for testing
    # df_scaled = df_scaled.tail(1000)  # Uncomment for testing

    import multiprocessing
    n_cpus = multiprocessing.cpu_count()

    # Set number of threads for Stan
    os.environ['STAN_NUM_THREADS'] = str(max(1, n_cpus - 1))

    print("Starting forecast...")
    model, forecast, prophet_df, cv_metrics, cv_results = create_prophet_forecast_with_validation(df_scaled)
    print("Forecast complete!")

    return model, forecast, cv_metrics, cv_results

if __name__ == "__main__":
    model, forecast, cv_metrics, cv_results = run_forecast(df_scaled)

Starting forecast...


16:36:14 - cmdstanpy - INFO - Chain [1] start processing
16:47:02 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import matplotlib.pyplot as plt
import numpy as np
import logging
import os

# Reduce logging noise
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

def create_prophet_forecast_with_validation(df_scaled, target_column='AQI', periods=24*7):
    """
    Create and train a Prophet model with cross-validation and detailed diagnostics.
    Optimized for faster execution.
    """
    # Set Stan backend options for faster computation
    os.environ['STAN_SAMPLING_ARGS'] = 'method=optimize algorithm=lbfgs iter=1000'

    # Prepare data for Prophet - use a subset for faster computation
    prophet_df = df_scaled.reset_index()
    prophet_df = prophet_df.rename(columns={
        'Datetime': 'ds',
        target_column: 'y'
    })

    # Create and train Prophet model with minimal computation settings
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10,
        holidays_prior_scale=10,
        mcmc_samples=0,  # Disable MCMC
        interval_width=0.95,
        n_changepoints=20,  # Reduce changepoints further
        growth='linear'  # Simpler growth model
    )

    # Only add strongest correlating features
    correlation_threshold = 0.3  # Increased threshold for fewer regressors
    important_columns = [col for col in df_scaled.columns
                        if col != target_column
                        and df_scaled[col].corr(df_scaled[target_column]) > correlation_threshold]

    for column in important_columns[:3]:  # Limit to top 3 regressors
        model.add_regressor(column)

    # Fit the model
    model.fit(prophet_df)

    # Perform cross validation with fewer iterations
    cv_results = cross_validation(
        model,
        initial='365.25 days',
        period='90 days',     # Increased period for fewer iterations
        horizon='7 days',
        parallel="processes",
        disable_tqdm=True    # Disable progress bar
    )

    # Calculate performance metrics
    cv_metrics = performance_metrics(cv_results)

    # Create future dataframe
    future = model.make_future_dataframe(periods=periods, freq='H')

    # Add regressor values efficiently
    for column in important_columns[:3]:
        future[column] = prophet_df[column].iloc[-1]

    # Make forecast
    forecast = model.predict(future)

    return model, forecast, prophet_df, cv_metrics, cv_results

# Example usage
def run_forecast(df_scaled):
    import multiprocessing
    n_cpus = multiprocessing.cpu_count()

    # Set number of threads for Stan
    os.environ['STAN_NUM_THREADS'] = str(max(1, n_cpus - 1))

    print("Starting forecast...")

    # Call the function and store returned values
    results = create_prophet_forecast_with_validation(df_scaled)

    # Check the number of returned values for debugging
    print(f"Number of returned values: {len(results)}")

    # Unpack results based on the number of returned values
    if len(results) == 5:
        model, forecast, prophet_df, cv_metrics, cv_results = results
    else:
        raise ValueError(f"Unexpected number of values returned: {len(results)}")

    print("Forecast complete!")

    # Print metrics
    print("\nCross-validation Metrics:")
    print(f"Mean Absolute Error (MAE): {cv_metrics['mae'].mean():.4f}")
    print(f"Root Mean Square Error (RMSE): {cv_metrics['rmse'].mean():.4f}")
    print(f"Mean Absolute Percentage Error (MAPE): {cv_metrics['mape'].mean():.2f}%")

    return model, forecast, cv_metrics, cv_results

# Run and print metrics
if __name__ == "__main__":
    # Assume df_scaled is already defined and contains the relevant data
    model, forecast, cv_metrics, cv_results = run_forecast(df_scaled)
